# A polymorphic value-type for C++

Jonathan B. Coe

<https://github.com/jbcoe/polymorphic_value/blob/master/draft.md>

The class template `polymorphic_value` is proposed for addition to the C++ standard Library.

A `polymorphic_value<T>` may hold an object of a class publicly derived from T.

Copying a `polymorphic_value<T>` will copy the object of the derived type.

## Component-based object design

Object orientated langauges let us design objects in terms of components:

```
class Composite {
  FooComponent f_; BarComponent b_;
  
 public:
  Composite(FooComponent f, BarComponent b) : f_(std::move(f)), b_(std::move(b)) {}
  
  void foo() const { f_.foo(); }
  void foo() { f_.foo(); }
    
  void bar() const { b_.bar(); }
  void bar() { b_.bar(); }
};
```


## Polymorphism

It would be more flexible to build composite objects out of sets of types.

* A `Zoo` could contain a list of `Animals` of different types.

* A `code_checker` could contain different `checking_tool`s.

* A `Game` could contain different `Entities`.

* A `Portfolio` could contain different kind of `FinancialInstrument`.

If a class contains objects of undecided type, it still needs to reserve storage for something.

### Closed-set polymorphism

When the number of different types is known at compile time, we can represent polymorphism with a variant:

```
using vehicle = std::variant<plane,train,automobile>;
```

`std::variant` will be part of C++17.

A variant keeps enough storage for its largest type and can use this storage to represent any of the types.

[ It's not identical to `std::variant` but I've found `eggs::variant` intuitive and fit-for-purpose <https://eggs-cpp.github.io/variant/>. ]

```
class ClosedSetComposite {
  using FooComponent = variant<Foo1, Foo2>;
  using BarComponent = variant<Bar1, Bar2, Bar3>;
  FooComponent f_; BarComponent b_;
  
 public:
  ClosedSetComposite(FooComponent f, BarComponent b) : f_(std::move(f)), b_(std::move(b)) {}
  
  void foo() const { visit(f_, [](const auto& x){x.foo();}); }
  void foo() { visit(f_, [](auto& x){x.foo();}); }
    
  void bar() const { visit(b_, [](const auto& x){x.bar();}); }
  void bar() { visit(b_, [](auto& x){x.bar();}); }
};
```

This will work nicely.

### Open-set polymorphism

When the number of different types is not known at compile time or is intentionally open for extension, we can represent polymorphism with inheritance:

```
struct Animal {
  virtual ~Animal() = default;
  virtual const char* make_noise() const =0;
};

struct Lion : Animal {
  virtual const char* make_noise() const override;
};
```

To store an polymorphic object using inheritance we typically use a pointer to the base type. 

The pointer will give us access to storage which can be allocated later.

Polymorphism through inheritance can localize code changes as type-specific behaviour can be handled by each type.

```
class OpenSetComposite {
  FooComponent* f_; BarComponent* b_;

 public:
  OpenSetComposite(FooComponent* f, BarComponent* b) : f_(f), b_(b) {}

  void foo() const { f_->foo(); }
  void foo() { f_->foo(); }

  void bar() const { b_->bar(); }
  void bar() { b_->bar(); }
};
```

This has several significant problems.

It compiles fine but it won't behave as we want it to.

## Smart pointers

Smart pointers have different semantics to raw pointers and can be used to express intent to the compiler.

* Ownership can be transferred.

* Resources can be freed on destruction.

`C++98` had `std::auto_ptr`. With the introduction of move semantics, we have improved smart pointers in C++11.

### `unique_ptr<T>`

```
class OpenSetComposite {
  std::unique_ptr<FooComponent> f_; std::unique_ptr<BarComponent> b_;

 public:
  OpenSetComposite(std::unique_ptr<FooComponent> f, 
                   std::unique_ptr<BarComponent> b) : f_(std::move(f)), b_(std::move(b)) {}

  void foo() const { f_->foo(); }
  void foo() { f_->foo(); }

  void bar() const { b_->bar(); }
  void bar() { b_->bar(); }
};
```

This is an improvement, we now have a correct compiler-generated destructor, move constructor and move assignement operator.

There are still shortcomings though, the object cannot be copied and the semantics are subtly different to our original design (all we want to do is add polymorphism to the components).

### const-propagation

```
struct SimpleFooComponent : FooComponent {
  void foo() const override { std::cout << "SimpleFooComponent::foo() const\n"; }
  void foo() override { std::cout << "SimpleFooComponent::foo()\n"; }
}

OpenSetComposite c(std::make_unique<SimpleFooComponent>(), /* omitted */ );
c.foo(); // prints "SimpleFooComponent::foo()"

const OpenSetComposite& cc = c;
cc.foo(); // prints "SimpleFooComponent::foo()"
```

The `const`-ness of the Composite is _not_ propagated to the component. 

This is how pointers, including smart pointers, work.

This _could_ be innocuous but silently changing semantics is a bug-in-waiting,

### `propagate_const<T>`

C++ Library Fundamentals Technical Specification 2 offers `propagate_const<T>` which will address the `const`-propagatation issue.

<http://en.cppreference.com/w/cpp/experimental/propagate_const>


```
template <typename T>
using member_ptr = std::experimental::propagate_const<std::unique_ptr<T>>;

class OpenSetComposite {
  member_ptr<FooComponent> f_; 
  member_ptr<BarComponent> b_;

 public:
  OpenSetComposite(std::unique_ptr<FooComponent> f, 
                   std::unique_ptr<BarComponent> b) : f_(std::move(f)), b_(std::move(b)) {}

  void foo() const { f_->foo(); }
  void foo() { f_->foo(); }

  void bar() const { b_->bar(); }
  void bar() { b_->bar(); }
};
```

`const`-ness of the composite object is now propagated to its components.

Our `OpenSetComposite` is still non-copyable though.

### `shared_ptr<const T>`

`shared_ptr<T>` as a member variable of a copyable object will introduce shared mutable state. We don't want that.

`shared_ptr<const T>` might work.

```
class OpenSetComposite {
  std::shared_ptr<const FooComponent> f_; 
  std::shared_ptr<const BarComponent> b_;

 public:
  OpenSetComposite(std::shared_ptr<const FooComponent> f, 
                   std::shared_ptr<const BarComponent> b) : f_(std::move(f)), b_(std::move(b)) {}

  void foo() const { f_->foo(); }

  void bar() const { b_->bar(); }
};
```

Compiler-generated destructor, copy-constructor, move-constructor, copy-assignment and move-assignment operators all work as intended.

We've lost mutability.

### `polymorphic_value<T>`

```
class OpenSetComposite {
  polymorphic_value<FooComponent> f_; 
  polymorphic_value<BarComponent> b_;

 public:
  OpenSetComposite(polymorphic_value<FooComponent> f, 
                   polymorphic_value<BarComponent> b) : f_(std::move(f)), b_(std::move(b)) {}

  void foo() const { f_->foo(); }
  void foo() { f_->foo(); }

  void bar() const { b_->bar(); }
  void bar() { b_->bar(); }
};
```

The compiler-generated destructor, copy-constructor, move-constructor, copy-assignment and move-assignment operators all work as intended.

We've correctly added polymorphism to our original component-based design.

## Design and implementation of `polymorphic_value<T>`

### `polymorphic_value`: Constructors

```
template <class T> class polymorphic_value { 

  polymorphic_value() noexcept;
  template <class U, class C=default_copy<U>, class D=default_delete<U>> 
    explicit polymorphic_value(U* p, C c=C{}, D d=D{}); // with restrictions

  polymorphic_value(const polymorphic_value& p);
  template <class U> polymorphic_value(const polymorphic_value<U>& p); // with restrictions
  template <class U> polymorphic_value(const U& u); // with restrictions

  polymorphic_value(polymorphic_value&& p) noexcept;
  template <class U> polymorphic_value(polymorphic_value<U>&& p); // with restrictions
  template <class U> polymorphic_value(U&& u); // with restrictions

```

### `polymorphic_value`: Assignment

```
  polymorphic_value &operator=(const polymorphic_value& p);
  template <class U> polymorphic_value &operator=(const polymorphic_value<U>& p); // with restrictions
  template <class U> polymorphic_value &operator=(const U& u); // with restrictions

  polymorphic_value &operator=(polymorphic_value &&p) noexcept;
  template <class U> polymorphic_value &operator=(polymorphic_value<U>&& p); // with restrictions
  template <class U> polymorphic_value &operator=(U&& u); // with restrictions
```

### `polymorphic_value`: Modifiers and observers
```
  void swap(polymorphic_value<T>& p) noexcept;

  T& operator*();
  T* operator->();
  const T& operator*() const;
  const T* operator->() const;
  explicit operator bool() const noexcept;
};
```

### `polymorphic_value`: Creation
```
template <class T, class ...Ts> polymorphic_value<T>
  make_polymorphic_value(Ts&& ...ts); 
```

This helper function allows more efficient construction of a `polymorphic_value` (like `make_shared` does for a `shared_ptr`).

## A reference implementation

### Type-erasure

## Feedback

The reference implementation is available on GitHub: <https://github.com/jbcoe/polymorphic_value>

It is MIT licensed with the intention of making it widely useable.

User-feedback is invited!

Please post GitHub issues with any user-feedback.